In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("from_postgres_to_clickhouse")
    .config(
        "spark.jars.packages",
        ",".join([
            # clickhouse drivers
            "com.clickhouse.spark:clickhouse-spark-runtime-3.4_2.12:0.8.0",
            "com.clickhouse:clickhouse-client:0.7.0",
            "com.clickhouse:clickhouse-http-client:0.7.0",
            "org.apache.httpcomponents.client5:httpclient5:5.2.1",
            # postgres drivers
            "org.postgresql:postgresql:42.7.3"
        ])
    )
    # clickhouse catalog conf
    .config("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
    .config("spark.sql.catalog.clickhouse.host", "clickhouse-server")
    .config("spark.sql.catalog.clickhouse.protocol", "http")
    .config("spark.sql.catalog.clickhouse.http_port", "8123")
    .config("spark.sql.catalog.clickhouse.user", "default")
    .config("spark.sql.catalog.clickhouse.password", "1234qwe")
    .config("spark.sql.catalog.clickhouse.database", "default")
    .config("spark.clickhouse.write.format", "json")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.clickhouse.spark#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-client added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a4104591-7281-4f2d-8151-2f5e5f7bb9d2;1.0
	confs: [default]
	found com.clickhouse.spark#clickhouse-spark-runtime-3.4_2.12;0.8.0 in central
	found com.clickhouse#clickhouse-client;0.7.0 in central
	found com.clickhouse#clickhouse-data;0.7.0 in central
	found com.clickhouse#clickhouse-http-client;0.7.0 in central
	found org.apache.httpcomponents.client5#httpclient5;5.2.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	

Get df from postgres

In [3]:
jdbc_url = "jdbc:postgresql://postgres:5432/postgres"
properties = {
    "user": "postgres_user",
    "password": "postgres_password",
    "driver": "org.postgresql.Driver"
}

df = spark.read.jdbc(url=jdbc_url, table="test_data", properties=properties)

df.show()

+---+-------+-----+
| id|   name|value|
+---+-------+-----+
|  1|  Alice|   10|
|  2|    Bob|   20|
|  3|Charlie|   30|
|  4|  Alice|   10|
|  5|    Bob|   20|
|  6|Charlie|   30|
+---+-------+-----+



Save df to clickhouse

In [4]:
spark.sql("""
CREATE TABLE IF NOT EXISTS clickhouse.default.test_table (
    id INT NOT NULL,
    name STRING,
    value INT
)
USING clickhouse
TBLPROPERTIES (
    'engine'='MergeTree()',
    'order_by'='id'
)
""")

DataFrame[]

In [5]:
df.writeTo("clickhouse.default.test_table").append()

In [6]:
df = spark.sql("select * from clickhouse.default.test_table")
df.show()

+---+-------+-----+
| id|   name|value|
+---+-------+-----+
|  1|  Alice|   10|
|  2|    Bob|   20|
|  3|Charlie|   30|
|  4|  Alice|   10|
|  5|    Bob|   20|
|  6|Charlie|   30|
+---+-------+-----+

